In [7]:
# First cell - Imports
import blpapi
import pandas as pd
import numpy as np
from datetime import datetime
import os
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [19]:
# Second cell - BloombergAnalyzer class
class BloombergAnalyzer:
    def __init__(self):
        self.session = None
        self._reference_data_service = None
        
    def connect(self) -> bool:
        """Establish connection to Bloomberg Terminal"""
        try:
            sessionOptions = blpapi.SessionOptions()
            sessionOptions.setServerHost("localhost")
            sessionOptions.setServerPort(8194)
            
            self.session = blpapi.Session(sessionOptions)
            
            if not self.session.start():
                logger.error("Failed to start session.")
                return False
                
            if not self.session.openService("//blp/refdata"):
                logger.error("Failed to open reference data service.")
                return False
                
            self._reference_data_service = self.session.getService("//blp/refdata")
            logger.info("Successfully connected to Bloomberg")
            return True
            
        except Exception as e:
            logger.error(f"Error connecting to Bloomberg: {str(e)}")
            return False
            
    def disconnect(self):
        """Disconnect from Bloomberg Terminal"""
        if self.session:
            self.session.stop()
            
    def get_bulk_fields_info_v2(self, securities):
        """Get and process bulk data fields with proper structure decomposition"""
        if isinstance(securities, str):
            securities = [securities]
            
        try:
            request = self._reference_data_service.createRequest("ReferenceDataRequest")
            
            # Add securities
            for security in securities:
                request.append("securities", security)
            
            # Bulk data fields grouped by type
            allocation_fields = {
                "HB_INDUSTRY_SECTOR_ALLOCATION": ["Sector", "Weight"],
                "FUND_SECTOR_ALLOCATION": ["Sector", "Weight"],
                "FUND_GEO_ALLOCATION": ["Country", "Weight"],
                "FUND_MARKET_CAP_BREAKDOWN": ["Range", "Weight"],
                "FUND_CURRENCY_BREAKDOWN": ["Currency", "Weight"],
                "FUND_REGION_BREAKDOWN": ["Region", "Weight"]
            }
            
            holdings_fields = {
                "FUND_TOP_HOLDINGS": ["Security", "Weight", "Market Value", "Shares"],
                "FUND_HOLDINGS": ["Security", "Weight", "Market Value", "Shares"]
            }
            
            fixed_income_fields = {
                "FUND_CREDIT_QUALITY_DIST": ["Rating", "Weight"],
                "FUND_MATURITY_BREAKDOWN": ["Range", "Weight"]
            }
            
            # Add all fields to request
            for field in {**allocation_fields, **holdings_fields, **fixed_income_fields}:
                request.append("fields", field)
                
            logger.info(f"Sending bulk data request for {len(securities)} securities")
            self.session.sendRequest(request)
            
            # Dictionary to store processed DataFrames
            processed_data = {
                "allocations": {},
                "holdings": {},
                "fixed_income": {}
            }
            
            # Process response
            while True:
                ev = self.session.nextEvent(500)
                
                for msg in ev:
                    if msg.hasElement("securityData"):
                        security_data = msg.getElement("securityData")
                        
                        for i in range(security_data.numValues()):
                            sec_data = security_data.getValueAsElement(i)
                            security = sec_data.getElementAsString("security")
                            
                            if sec_data.hasElement("fieldData"):
                                field_data = sec_data.getElement("fieldData")
                                
                                # Process allocation fields
                                for field, columns in allocation_fields.items():
                                    if field_data.hasElement(field):
                                        try:
                                            data_list = []
                                            bulk_element = field_data.getElement(field)
                                            
                                            for j in range(bulk_element.numValues()):
                                                value_element = bulk_element.getValueAsElement(j)
                                                row_data = {}
                                                for col in columns:
                                                    if value_element.hasElement(col):
                                                        row_data[col] = value_element.getElement(col).getValue()
                                                data_list.append(row_data)
                                                
                                            if data_list:
                                                df = pd.DataFrame(data_list)
                                                df['Security'] = security
                                                processed_data['allocations'][field] = df
                                                
                                                # Save individual allocation to CSV
                                                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                                                filename = f"data/{field.lower()}_{timestamp}.csv"
                                                df.to_csv(filename, index=False)
                                                logger.info(f"Saved {field} data to {filename}")
                                                
                                        except Exception as e:
                                            logger.error(f"Error processing {field}: {str(e)}")
                                            
                                # Process holdings fields similarly
                                for field, columns in holdings_fields.items():
                                    # Similar processing as above but for holdings
                                    pass
                                    
                                # Process fixed income fields similarly
                                for field, columns in fixed_income_fields.items():
                                    # Similar processing as above but for fixed income
                                    pass
                                    
                if ev.eventType() == blpapi.Event.RESPONSE:
                    break
                    
            return processed_data
            
        except Exception as e:
            logger.error(f"Error getting bulk field info: {str(e)}")
            return {}

    
    
    def get_bulk_fields_info(self, securities):
        """Get bulk data fields that return nested/array data"""
        if isinstance(securities, str):
            securities = [securities]
            
        try:
            request = self._reference_data_service.createRequest("ReferenceDataRequest")
            
            # Add securities
            for security in securities:
                request.append("securities", security)
            
            # Bulk data fields that return nested structures
            bulk_fields = [
                "HB_INDUSTRY_SECTOR_ALLOCATION",  # Industry Sector Allocation
                "FUND_SECTOR_ALLOCATION",         # Fund Sector Allocation
                "FUND_GEO_ALLOCATION",            # Geographic Allocation
                "FUND_HOLDING_CLASS_LONG",        # Asset Class Allocation
                "FUND_HOLDINGS",                  # Fund Holdings
                "FUND_TOP_HOLDINGS",              # Top Holdings
                "FUND_CREDIT_QUALITY_DIST",       # Credit Quality Distribution
                "FUND_MATURITY_BREAKDOWN",        # Maturity Breakdown
                "FUND_MARKET_CAP_BREAKDOWN",      # Market Cap Breakdown
                "FUND_CURRENCY_BREAKDOWN",        # Currency Breakdown
                "FUND_REGION_BREAKDOWN",          # Regional Breakdown
            ]
            
            for field in bulk_fields:
                request.append("fields", field)
                
            logger.info(f"Sending bulk data request for {len(securities)} securities")
            self.session.sendRequest(request)
            
            # Process response
            data = []
            while True:
                ev = self.session.nextEvent(500)
                
                for msg in ev:
                    if msg.hasElement("securityData"):
                        security_data = msg.getElement("securityData")
                        
                        for i in range(security_data.numValues()):
                            sec_data = security_data.getValueAsElement(i)
                            security = sec_data.getElementAsString("security")
                            
                            # Initialize dictionary for this security
                            sec_info = {"security": security}
                            
                            # Get field data if available
                            if sec_data.hasElement("fieldData"):
                                field_data = sec_data.getElement("fieldData")
                                
                                # Process each bulk field
                                for field in bulk_fields:
                                    if field_data.hasElement(field):
                                        try:
                                            bulk_element = field_data.getElement(field)
                                            
                                            # Handle bulk data based on field type
                                            if bulk_element.isArray():
                                                field_values = []
                                                
                                                # Iterate through bulk data array
                                                for j in range(bulk_element.numValues()):
                                                    value_element = bulk_element.getValueAsElement(j)
                                                    item_data = {}
                                                    
                                                    # Get all sub-elements
                                                    for k in range(value_element.numElements()):
                                                        sub_element = value_element.getElement(k)
                                                        item_data[sub_element.name()] = sub_element.getValue()
                                                        
                                                    field_values.append(item_data)
                                                    
                                                sec_info[field] = field_values
                                        except Exception as e:
                                            logger.error(f"Error processing bulk field {field}: {str(e)}")
                                            sec_info[field] = None
                                    else:
                                        sec_info[field] = None
                                        
                            data.append(sec_info)
                
                if ev.eventType() == blpapi.Event.RESPONSE:
                    break
                    
            # Convert to DataFrame with nested data
            df = pd.DataFrame(data)
            
            # Save to CSV (note: nested data will be stored as strings)
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"bulk_data_{timestamp}.csv"
            df.to_csv(filename, index=False)
            logger.info(f"Bulk data saved to {filename}")
            
            return df
            
        except Exception as e:
            logger.error(f"Error getting bulk field info: {str(e)}")
            return pd.DataFrame()
    
    def get_security_info(self, securities, requested_fields):
        """Get comprehensive security information"""
        if isinstance(securities, str):
            securities = [securities]
            
        try:
            request = self._reference_data_service.createRequest("ReferenceDataRequest")
            
            # Add securities
            for security in securities:
                request.append("securities", security)

                         
            for field in requested_fields:
                request.append("fields", field)
                
            logger.info(f"Sending request for {len(securities)} securities")
            self.session.sendRequest(request)
            
            # Process response
            data = []
            while True:
                ev = self.session.nextEvent(500)
                
                for msg in ev:
                    if msg.hasElement("securityData"):
                        security_data = msg.getElement("securityData")
                        
                        for i in range(security_data.numValues()):
                            sec_data = security_data.getValueAsElement(i)
                            security = sec_data.getElementAsString("security")
                            
                            # Initialize dictionary for this security
                            sec_info = {"security": security}
                            
                            # Get field data if available
                            if sec_data.hasElement("fieldData"):
                                field_data = sec_data.getElement("fieldData")
                                
                                # Extract each field value
                                for field in requested_fields:
                                    if field_data.hasElement(field):
                                        try:
                                            value = field_data.getElement(field).getValue()
                                            sec_info[field] = value
                                        except:
                                            sec_info[field] = None
                                    else:
                                        sec_info[field] = None
                                        
                            data.append(sec_info)
                
                if ev.eventType() == blpapi.Event.RESPONSE:
                    break
                    
            # Convert to DataFrame
            df = pd.DataFrame(data)
            
            # Save to CSV
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"security_data_{timestamp}.csv"
            df.to_csv(filename, index=False)
            logger.info(f"Data saved to {filename}")
            
            return df
            
        except Exception as e:
            logger.error(f"Error getting security info: {str(e)}")
            return pd.DataFrame()

In [9]:
# Basic Security Information
basic_fields = [
    "SECURITY_TYP",        # Asset Type
    "SECURITY_TYP2",       # Detailed Asset Type
    "MARKET_SECTOR",       # Market Sector
    "SECURITY_NAME",       # Security Name
    "NAME",                # Company Name
    "SHORT_NAME",          # Short Name
    "ID_ISIN",            # ISIN
    "TICKER",              # Ticker
    "EQY_PRIM_EXCH",      # Primary Exchange
]

# Classification Fields
classification_fields = [
    "GICS_SECTOR_NAME",    # GICS Sector
    "GICS_INDUSTRY_NAME",  # GICS Industry
    "GICS_SUB_INDUSTRY",   # GICS Sub-Industry
    "INDUSTRY_GROUP",      # Bloomberg Industry Group
    "INDUSTRY_SECTOR",     # Industry Sector
    "INDUSTRY_SUBGROUP"    # Industry Subgroup
]

# Geographic Fields
geographic_fields = [
    "CNTRY_OF_RISK",       # Country of Risk
    "CNTRY_OF_DOMICILE",   # Country of Domicile
    "CNTRY_OF_INCORPORATION" # Country of Incorporation
]

# Risk & Size Metrics
risk_fields = [
    "MARKET_CAP",          # Market Cap
    "CUR_MKT_CAP",        # Current Market Cap
    "VOLUME_AVG_30D",      # 30-day Average Volume
    "VOLATILITY_30D",      # 30-day Volatility
    "BETA_ADJ_OVERRIDABLE", # Beta
    "RSK_CURRENCY_RISK"    # Currency Risk
]

# Fixed Income Specific Fields
fixed_income_fields = [
    "MATURITY",            # Maturity Date
    "CPN",                 # Coupon
    "YLD_YTM_MID",         # Yield to Maturity
    "DURATION_MID",        # Duration
    "SECURITY_DES"         # Security Description
]

# ESG Fields
esg_fields = [
    "ESG_DISCLOSURE_SCORE",  # ESG Score
    "SUSTAINALYTICS_RANK"    # Sustainalytics Rank
]

# Add all fields to request
all_equity_fields = (basic_fields + classification_fields + geographic_fields + 
                risk_fields + fixed_income_fields + esg_fields)

In [10]:
# Fund Information Fields
fund_info_fields = [
    "FUND_TICKER",           # Fund Ticker
    "FUND_NAME",            # Fund Name
    "ID_ISIN",              # ISIN
    "FUND_ASSET_CLASS",     # Primary Asset Class
    "FUND_ASSET_CLASS_FOCUS", # Specific Asset Class Focus
    "FUND_TYPE",            # Fund Type (ETF/Mutual Fund)
    "FUND_STATUS",          # Active/Inactive Status
    "FUND_INCEPT_DT",       # Inception Date
    "FUND_ASSET_CLASS_FOCUS", # Asset Class Focus
    "FUND_GEOGRAPHIC_FOCUS", # Geographic Focus
    "FUND_INDUSTRY_FOCUS",  # Industry Focus
    "FUND_MGT_CO",          # Management Company
    "FUND_ADMIN",           # Fund Administrator
    "FUND_DEALING_STRUCTURE", # Dealing Structure
    "FUND_LEVERAGE_MULTIPLE", # Leverage Multiple if applicable
]

# Asset Allocation
allocation_fields = [
    "FUND_ASSET_ALLOC_EQUITY",    # Equity Allocation
    "FUND_ASSET_ALLOC_FIXED_INCOME", # Fixed Income Allocation
    "FUND_ASSET_ALLOC_CASH",      # Cash Allocation
    "FUND_ASSET_ALLOC_OTHER",     # Other Assets Allocation
    "FUND_SECTOR_ALLOCATION",     # Sector Allocation
    "FUND_GEO_ALLOCATION",        # Geographic Allocation
    "FUND_MARKET_CAP_FOCUS",      # Market Cap Focus
    "FUND_STYLE_BOX",            # Investment Style Box
    "TOP_10_HOLDINGS_PCT",        # Top 10 Holdings %
    "TOP_20_HOLDINGS_PCT",        # Top 20 Holdings %
    "NUM_HOLDINGS",               # Number of Holdings
    "PCT_INVESTED_TOP_10",        # % in Top 10 Holdings
]

# Performance Metrics
performance_fields = [
    "FUND_NET_ASSET_VAL",        # NAV
    "FUND_TOTAL_ASSETS",         # Total Assets
    "FUND_NET_FLOW",             # Fund Flow
    "TOT_RETURN_INDEX_GROSS_DVDS", # Total Return Index
    "FUND_TOTAL_RETURNS_1M",      # 1 Month Return
    "FUND_TOTAL_RETURNS_3M",      # 3 Month Return
    "FUND_TOTAL_RETURNS_6M",      # 6 Month Return
    "FUND_TOTAL_RETURNS_1Y",      # 1 Year Return
    "FUND_TOTAL_RETURNS_3Y",      # 3 Year Return
    "FUND_TOTAL_RETURNS_5Y",      # 5 Year Return
    "FUND_TOTAL_RETURNS_10Y",     # 10 Year Return
    "FUND_PERF_TRK_REC",         # Performance Track Record
    "YTD_RETURN",                # Year to Date Return
    "FUND_VOL_R1Y_ANN",          # 1Y Annualized Volatility
    "FUND_VOL_R3Y_ANN",          # 3Y Annualized Volatility
    "FUND_VOL_R5Y_ANN",          # 5Y Annualized Volatility
]

# Risk Metrics
risk_fields = [
    "FUND_SHARPE_RATIO_1YR",     # 1Y Sharpe Ratio
    "FUND_SHARPE_RATIO_3YR",     # 3Y Sharpe Ratio
    "FUND_SHARPE_RATIO_5YR",     # 5Y Sharpe Ratio
    "FUND_SORTINO_RATIO_1YR",    # 1Y Sortino Ratio
    "FUND_SORTINO_RATIO_3YR",    # 3Y Sortino Ratio
    "FUND_INFO_RATIO_1YR",       # 1Y Information Ratio
    "FUND_INFO_RATIO_3YR",       # 3Y Information Ratio
    "FUND_ALPHA_1YR",            # 1Y Alpha
    "FUND_ALPHA_3YR",            # 3Y Alpha
    "FUND_BETA_1YR",             # 1Y Beta
    "FUND_BETA_3YR",             # 3Y Beta
    "FUND_R_SQUARED_1YR",        # 1Y R-Squared
    "FUND_R_SQUARED_3YR",        # 3Y R-Squared
    "MAX_DRAWDOWN_1YR",          # 1Y Maximum Drawdown
    "MAX_DRAWDOWN_3YR",          # 3Y Maximum Drawdown
    "FUND_STD_DEV_3YR",          # 3Y Standard Deviation
    "FUND_UP_CAPTURE_3YR",       # 3Y Up Market Capture
    "FUND_DOWN_CAPTURE_3YR",     # 3Y Down Market Capture
    "FUND_TRACK_ERROR_1YR",      # 1Y Tracking Error
    "FUND_TRACK_ERROR_3YR",      # 3Y Tracking Error
]

# Fee Structure
fee_fields = [
    "FUND_TOTAL_EXPENSE_RATIO",  # Total Expense Ratio
    "FUND_NET_EXP_RATIO",        # Net Expense Ratio
    "FUND_GROSS_EXP_RATIO",      # Gross Expense Ratio
    "FUND_MGT_FEE",              # Management Fee
    "FUND_PERFORMANCE_FEE",      # Performance Fee
    "FUND_ADMIN_FEE",            # Administrative Fee
    "FUND_CUSTODY_FEE",          # Custody Fee
    "FUND_DISTRIBUTION_FEE",     # Distribution Fee
    "FUND_INITIAL_CHARGE",       # Initial Charge
    "FUND_EXIT_FEE",             # Exit Fee
    "FUND_REDEMPTION_FEE",       # Redemption Fee
    "FUND_SWITCHING_FEE",        # Switching Fee
]

# Trading Information
trading_fields = [
    "FUND_MIN_INVEST",           # Minimum Investment
    "FUND_MIN_SUBSEQUENT_INVEST", # Minimum Subsequent Investment
    "FUND_SETTLEMENT_DT",        # Settlement Date
    "FUND_DEALING_FREQUENCY",    # Dealing Frequency
    "FUND_SUBSCRIPTION_TYPE",    # Subscription Type
    "FUND_REDEMPTION_TYPE",      # Redemption Type
    "PX_LAST",                   # Last Price
    "PX_HIGH_52WEEK",           # 52 Week High
    "PX_LOW_52WEEK",            # 52 Week Low
    "VOLUME_AVG_30D",           # 30 Day Average Volume
    "VOLUME_AVG_20D",           # 20 Day Average Volume
    "FUND_ASSETS_DAILY",        # Daily Assets
    "FUND_ASSETS_WEEKLY",       # Weekly Assets
]

# Dividend/Income Information
dividend_fields = [
    "FUND_DIVIDEND_FREQUENCY",   # Dividend Frequency
    "FUND_DIVIDEND_TYPE",        # Dividend Type (Accumulating/Distributing)
    "FUND_DIVIDEND_CURRENCY",    # Dividend Currency
    "FUND_INCOME_CURRENCY",      # Income Currency
    "FUND_DIVIDEND_YIELD",       # Dividend Yield
    "FUND_EST_DIVIDEND_YIELD",   # Estimated Dividend Yield
    "FUND_HISTORICAL_YIELD",     # Historical Yield
    "FUND_DISTRIBUTION_POLICY",  # Distribution Policy
]

# ETF Specific Fields
etf_fields = [
    "ETF_CREATION_UNIT_SIZE",    # Creation Unit Size
    "ETF_PRIMARY_INDEX",         # Primary Tracking Index
    "ETF_TRACKING_DIFFERENCE",   # Tracking Difference
    "ETF_REPLICATION_STRUCTURE", # Replication Structure (Physical/Synthetic)
    "ETF_SWAP_COUNTERPARTY",     # Swap Counterparty if applicable
    "ETF_SECURITIES_LENDING",    # Securities Lending Permitted
    "ETF_SECURITIES_LENT_PCT",   # % of Securities Lent
    "ETF_COLLATERAL_TYPE",       # Collateral Type
    "ETF_OPTIONS_AVAILABLE",     # Options Available
    "ETF_SHORT_INTEREST_PCT",    # Short Interest %
    "ETF_HEDGE_TYPE",           # Currency Hedge Type
]

# Fixed Income Fund Specific
fixed_income_fields = [
    "FUND_CREDIT_QUALITY",       # Credit Quality Breakdown
    "FUND_EFFECTIVE_DURATION",   # Effective Duration
    "FUND_AVG_MATURITY",        # Average Maturity
    "FUND_AVG_COUPON",          # Average Coupon
    "FUND_YTM",                 # Yield to Maturity
    "FUND_DURATION_ADJ_YLD",    # Duration Adjusted Yield
    "FUND_CREDIT_RATING",       # Average Credit Rating
    "FUND_RATING_STATS",        # Rating Statistics
    "FUND_INT_RATE_RISK",       # Interest Rate Risk
]

# ESG Information
esg_fields = [
    "ESG_DISCLOSURE_SCORE",      # ESG Disclosure Score
    "FUND_ESG_SCORE",           # Fund ESG Score
    "FUND_ESG_RATING",          # Fund ESG Rating
    "FUND_CARBON_INTENSITY",    # Carbon Intensity
    "FUND_SUSTAINABILITY_RATING", # Sustainability Rating
    "FUND_ESG_COVERAGE_PCT"     # ESG Coverage %
]

# Additional Analytics
analytics_fields = [
    "FUND_PEER_GROUP_RANK_1YR", # 1Y Peer Group Ranking
    "FUND_PEER_GROUP_RANK_3YR", # 3Y Peer Group Ranking
    "FUND_PEER_GROUP_RANK_5YR", # 5Y Peer Group Ranking
    "FUND_CATEGORY_BENCHMARK",   # Category Benchmark
    "FUND_GROWTH_OF_10K",       # Growth of 10K
    "FUND_TURNOVER_RATIO",      # Portfolio Turnover Ratio
    "FUND_ACTIVE_SHARE",        # Active Share
    "FUND_PRICE_MOMENTUM",      # Price Momentum
    "FUND_PEER_CORRELATION",    # Peer Correlation
]

flo = [
"NAME",
"SHORT_NAME",
"SECURITY_NAME",
"COUNTRY_FULL_NAME",
"PRIMARY_EXCHANGE_NAME",
"BICS_LEVEL_1_SECTOR_NAME",
"ICB_SUPERSECTOR_NAME",
"ICB_SECTOR_NAME",
"BICS_LEVEL_2_INDUSTRY_GROUP_NAME",
"INDUSTRY_SECTOR",
"MARKET_SECTOR_DES",
"INDUSTRY_SECTOR_RT",
"INDUSTRY_GROUP",
"INDUSTRY_SUBGROUP",
"HB_INDUSTRY_SECTOR_ALLOCATION"








]


# Combine all fields
all_fund_fields = (fund_info_fields + allocation_fields + performance_fields + 
             risk_fields + fee_fields + trading_fields + dividend_fields + 
             etf_fields + fixed_income_fields + esg_fields + analytics_fields)

In [12]:
# Third cell - Example usage
analyzer = BloombergAnalyzer()

try:
    if analyzer.connect():
        # Test with some sample securities
        securities = [
            # "AAPL US Equity",
            # "MSFT US Equity",
            # "IBM US Equity",
            # "TSLA US Equity"
            "CSPX LN Equity"
        ]
        
        # Get security information
        df = analyzer.get_security_info(securities, all_fund_fields)
        
        # Display the first few rows
        print("\nSecurity Information:")
        print(df.head())
        
        # Display some basic statistics
        print("\nBasic Statistics:")
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        print(df[numeric_cols].describe())
        
    
     
        
finally:
    analyzer.disconnect()

INFO:__main__:Successfully connected to Bloomberg
INFO:__main__:Sending request for 1 securities
INFO:__main__:Data saved to security_data_20250121_183443.csv



Security Information:
         security FUND_TICKER FUND_NAME       ID_ISIN FUND_ASSET_CLASS  \
0  CSPX LN Equity        None      None  IE00B5BMR087             None   

  FUND_ASSET_CLASS_FOCUS FUND_TYPE FUND_STATUS FUND_INCEPT_DT  \
0                 Equity      None        None     2010-09-15   

  FUND_GEOGRAPHIC_FOCUS  ... FUND_ESG_COVERAGE_PCT FUND_PEER_GROUP_RANK_1YR  \
0                  None  ...                  None                     None   

  FUND_PEER_GROUP_RANK_3YR FUND_PEER_GROUP_RANK_5YR FUND_CATEGORY_BENCHMARK  \
0                     None                     None                    None   

   FUND_GROWTH_OF_10K FUND_TURNOVER_RATIO FUND_ACTIVE_SHARE  \
0                None                None              None   

  FUND_PRICE_MOMENTUM FUND_PEER_CORRELATION  
0                None                  None  

[1 rows x 131 columns]

Basic Statistics:
       FUND_ASSET_ALLOC_EQUITY  FUND_NET_ASSET_VAL  FUND_TOTAL_ASSETS  \
count                  1.00000              

In [ ]:
# Fund Information Fields
fund_info_fields = [
    "FUND_TICKER",           # Fund Ticker
    "FUND_NAME",            # Fund Name
    "ID_ISIN",              # ISIN
    "FUND_ASSET_CLASS",     # Primary Asset Class
    "FUND_ASSET_CLASS_FOCUS", # Specific Asset Class Focus
    "FUND_TYPE",            # Fund Type (ETF/Mutual Fund)
    "FUND_STATUS",          # Active/Inactive Status
    "FUND_INCEPT_DT",       # Inception Date
    "FUND_ASSET_CLASS_FOCUS", # Asset Class Focus
    "FUND_GEOGRAPHIC_FOCUS", # Geographic Focus
    "FUND_INDUSTRY_FOCUS",  # Industry Focus
    "FUND_MGT_CO",          # Management Company
    "FUND_ADMIN",           # Fund Administrator
    "FUND_DEALING_STRUCTURE", # Dealing Structure
    "FUND_LEVERAGE_MULTIPLE", # Leverage Multiple if applicable
]

# Asset Allocation
allocation_fields = [
    "FUND_ASSET_ALLOC_EQUITY",    # Equity Allocation
    "FUND_ASSET_ALLOC_FIXED_INCOME", # Fixed Income Allocation
    "FUND_ASSET_ALLOC_CASH",      # Cash Allocation
    "FUND_ASSET_ALLOC_OTHER",     # Other Assets Allocation
    "FUND_SECTOR_ALLOCATION",     # Sector Allocation
    "FUND_GEO_ALLOCATION",        # Geographic Allocation
    "FUND_MARKET_CAP_FOCUS",      # Market Cap Focus
    "FUND_STYLE_BOX",            # Investment Style Box
    "TOP_10_HOLDINGS_PCT",        # Top 10 Holdings %
    "TOP_20_HOLDINGS_PCT",        # Top 20 Holdings %
    "NUM_HOLDINGS",               # Number of Holdings
    "PCT_INVESTED_TOP_10",        # % in Top 10 Holdings
]

# Performance Metrics
performance_fields = [
    "FUND_NET_ASSET_VAL",        # NAV
    "FUND_TOTAL_ASSETS",         # Total Assets
    "FUND_NET_FLOW",             # Fund Flow
    "TOT_RETURN_INDEX_GROSS_DVDS", # Total Return Index
    "FUND_TOTAL_RETURNS_1M",      # 1 Month Return
    "FUND_TOTAL_RETURNS_3M",      # 3 Month Return
    "FUND_TOTAL_RETURNS_6M",      # 6 Month Return
    "FUND_TOTAL_RETURNS_1Y",      # 1 Year Return
    "FUND_TOTAL_RETURNS_3Y",      # 3 Year Return
    "FUND_TOTAL_RETURNS_5Y",      # 5 Year Return
    "FUND_TOTAL_RETURNS_10Y",     # 10 Year Return
    "FUND_PERF_TRK_REC",         # Performance Track Record
    "YTD_RETURN",                # Year to Date Return
    "FUND_VOL_R1Y_ANN",          # 1Y Annualized Volatility
    "FUND_VOL_R3Y_ANN",          # 3Y Annualized Volatility
    "FUND_VOL_R5Y_ANN",          # 5Y Annualized Volatility
]

# Risk Metrics
risk_fields = [
    "FUND_SHARPE_RATIO_1YR",     # 1Y Sharpe Ratio
    "FUND_SHARPE_RATIO_3YR",     # 3Y Sharpe Ratio
    "FUND_SHARPE_RATIO_5YR",     # 5Y Sharpe Ratio
    "FUND_SORTINO_RATIO_1YR",    # 1Y Sortino Ratio
    "FUND_SORTINO_RATIO_3YR",    # 3Y Sortino Ratio
    "FUND_INFO_RATIO_1YR",       # 1Y Information Ratio
    "FUND_INFO_RATIO_3YR",       # 3Y Information Ratio
    "FUND_ALPHA_1YR",            # 1Y Alpha
    "FUND_ALPHA_3YR",            # 3Y Alpha
    "FUND_BETA_1YR",             # 1Y Beta
    "FUND_BETA_3YR",             # 3Y Beta
    "FUND_R_SQUARED_1YR",        # 1Y R-Squared
    "FUND_R_SQUARED_3YR",        # 3Y R-Squared
    "MAX_DRAWDOWN_1YR",          # 1Y Maximum Drawdown
    "MAX_DRAWDOWN_3YR",          # 3Y Maximum Drawdown
    "FUND_STD_DEV_3YR",          # 3Y Standard Deviation
    "FUND_UP_CAPTURE_3YR",       # 3Y Up Market Capture
    "FUND_DOWN_CAPTURE_3YR",     # 3Y Down Market Capture
    "FUND_TRACK_ERROR_1YR",      # 1Y Tracking Error
    "FUND_TRACK_ERROR_3YR",      # 3Y Tracking Error
]

# Fee Structure
fee_fields = [
    "FUND_TOTAL_EXPENSE_RATIO",  # Total Expense Ratio
    "FUND_NET_EXP_RATIO",        # Net Expense Ratio
    "FUND_GROSS_EXP_RATIO",      # Gross Expense Ratio
    "FUND_MGT_FEE",              # Management Fee
    "FUND_PERFORMANCE_FEE",      # Performance Fee
    "FUND_ADMIN_FEE",            # Administrative Fee
    "FUND_CUSTODY_FEE",          # Custody Fee
    "FUND_DISTRIBUTION_FEE",     # Distribution Fee
    "FUND_INITIAL_CHARGE",       # Initial Charge
    "FUND_EXIT_FEE",             # Exit Fee
    "FUND_REDEMPTION_FEE",       # Redemption Fee
    "FUND_SWITCHING_FEE",        # Switching Fee
]

# Trading Information
trading_fields = [
    "FUND_MIN_INVEST",           # Minimum Investment
    "FUND_MIN_SUBSEQUENT_INVEST", # Minimum Subsequent Investment
    "FUND_SETTLEMENT_DT",        # Settlement Date
    "FUND_DEALING_FREQUENCY",    # Dealing Frequency
    "FUND_SUBSCRIPTION_TYPE",    # Subscription Type
    "FUND_REDEMPTION_TYPE",      # Redemption Type
    "PX_LAST",                   # Last Price
    "PX_HIGH_52WEEK",           # 52 Week High
    "PX_LOW_52WEEK",            # 52 Week Low
    "VOLUME_AVG_30D",           # 30 Day Average Volume
    "VOLUME_AVG_20D",           # 20 Day Average Volume
    "FUND_ASSETS_DAILY",        # Daily Assets
    "FUND_ASSETS_WEEKLY",       # Weekly Assets
]

# Dividend/Income Information
dividend_fields = [
    "FUND_DIVIDEND_FREQUENCY",   # Dividend Frequency
    "FUND_DIVIDEND_TYPE",        # Dividend Type (Accumulating/Distributing)
    "FUND_DIVIDEND_CURRENCY",    # Dividend Currency
    "FUND_INCOME_CURRENCY",      # Income Currency
    "FUND_DIVIDEND_YIELD",       # Dividend Yield
    "FUND_EST_DIVIDEND_YIELD",   # Estimated Dividend Yield
    "FUND_HISTORICAL_YIELD",     # Historical Yield
    "FUND_DISTRIBUTION_POLICY",  # Distribution Policy
]

# ETF Specific Fields
etf_fields = [
    "ETF_CREATION_UNIT_SIZE",    # Creation Unit Size
    "ETF_PRIMARY_INDEX",         # Primary Tracking Index
    "ETF_TRACKING_DIFFERENCE",   # Tracking Difference
    "ETF_REPLICATION_STRUCTURE", # Replication Structure (Physical/Synthetic)
    "ETF_SWAP_COUNTERPARTY",     # Swap Counterparty if applicable
    "ETF_SECURITIES_LENDING",    # Securities Lending Permitted
    "ETF_SECURITIES_LENT_PCT",   # % of Securities Lent
    "ETF_COLLATERAL_TYPE",       # Collateral Type
    "ETF_OPTIONS_AVAILABLE",     # Options Available
    "ETF_SHORT_INTEREST_PCT",    # Short Interest %
    "ETF_HEDGE_TYPE",           # Currency Hedge Type
]

# Fixed Income Fund Specific
fixed_income_fields = [
    "FUND_CREDIT_QUALITY",       # Credit Quality Breakdown
    "FUND_EFFECTIVE_DURATION",   # Effective Duration
    "FUND_AVG_MATURITY",        # Average Maturity
    "FUND_AVG_COUPON",          # Average Coupon
    "FUND_YTM",                 # Yield to Maturity
    "FUND_DURATION_ADJ_YLD",    # Duration Adjusted Yield
    "FUND_CREDIT_RATING",       # Average Credit Rating
    "FUND_RATING_STATS",        # Rating Statistics
    "FUND_INT_RATE_RISK",       # Interest Rate Risk
]

# ESG Information
esg_fields = [
    "ESG_DISCLOSURE_SCORE",      # ESG Disclosure Score
    "FUND_ESG_SCORE",           # Fund ESG Score
    "FUND_ESG_RATING",          # Fund ESG Rating
    "FUND_CARBON_INTENSITY",    # Carbon Intensity
    "FUND_SUSTAINABILITY_RATING", # Sustainability Rating
    "FUND_ESG_COVERAGE_PCT"     # ESG Coverage %
]

# Additional Analytics
analytics_fields = [
    "FUND_PEER_GROUP_RANK_1YR", # 1Y Peer Group Ranking
    "FUND_PEER_GROUP_RANK_3YR", # 3Y Peer Group Ranking
    "FUND_PEER_GROUP_RANK_5YR", # 5Y Peer Group Ranking
    "FUND_CATEGORY_BENCHMARK",   # Category Benchmark
    "FUND_GROWTH_OF_10K",       # Growth of 10K
    "FUND_TURNOVER_RATIO",      # Portfolio Turnover Ratio
    "FUND_ACTIVE_SHARE",        # Active Share
    "FUND_PRICE_MOMENTUM",      # Price Momentum
    "FUND_PEER_CORRELATION",    # Peer Correlation
]

# Combine all fields
all_fund_fields = (fund_info_fields + allocation_fields + performance_fields + 
             risk_fields + fee_fields + trading_fields + dividend_fields + 
             etf_fields + fixed_income_fields + esg_fields + analytics_fields)

In [13]:
def get_bulk_fields_info(self, securities):
    """Get bulk data fields that return nested/array data"""
    if isinstance(securities, str):
        securities = [securities]
        
    try:
        request = self._reference_data_service.createRequest("ReferenceDataRequest")
        
        # Add securities
        for security in securities:
            request.append("securities", security)
        
        # Bulk data fields that return nested structures
        bulk_fields = [
            "HB_INDUSTRY_SECTOR_ALLOCATION",  # Industry Sector Allocation
            "FUND_SECTOR_ALLOCATION",         # Fund Sector Allocation
            "FUND_GEO_ALLOCATION",            # Geographic Allocation
            "FUND_HOLDING_CLASS_LONG",        # Asset Class Allocation
            "FUND_HOLDINGS",                  # Fund Holdings
            "FUND_TOP_HOLDINGS",              # Top Holdings
            "FUND_CREDIT_QUALITY_DIST",       # Credit Quality Distribution
            "FUND_MATURITY_BREAKDOWN",        # Maturity Breakdown
            "FUND_MARKET_CAP_BREAKDOWN",      # Market Cap Breakdown
            "FUND_CURRENCY_BREAKDOWN",        # Currency Breakdown
            "FUND_REGION_BREAKDOWN",          # Regional Breakdown
        ]
        
        for field in bulk_fields:
            request.append("fields", field)
            
        logger.info(f"Sending bulk data request for {len(securities)} securities")
        self.session.sendRequest(request)
        
        # Process response
        data = []
        while True:
            ev = self.session.nextEvent(500)
            
            for msg in ev:
                if msg.hasElement("securityData"):
                    security_data = msg.getElement("securityData")
                    
                    for i in range(security_data.numValues()):
                        sec_data = security_data.getValueAsElement(i)
                        security = sec_data.getElementAsString("security")
                        
                        # Initialize dictionary for this security
                        sec_info = {"security": security}
                        
                        # Get field data if available
                        if sec_data.hasElement("fieldData"):
                            field_data = sec_data.getElement("fieldData")
                            
                            # Process each bulk field
                            for field in bulk_fields:
                                if field_data.hasElement(field):
                                    try:
                                        bulk_element = field_data.getElement(field)
                                        
                                        # Handle bulk data based on field type
                                        if bulk_element.isArray():
                                            field_values = []
                                            
                                            # Iterate through bulk data array
                                            for j in range(bulk_element.numValues()):
                                                value_element = bulk_element.getValueAsElement(j)
                                                item_data = {}
                                                
                                                # Get all sub-elements
                                                for k in range(value_element.numElements()):
                                                    sub_element = value_element.getElement(k)
                                                    item_data[sub_element.name()] = sub_element.getValue()
                                                    
                                                field_values.append(item_data)
                                                
                                            sec_info[field] = field_values
                                    except Exception as e:
                                        logger.error(f"Error processing bulk field {field}: {str(e)}")
                                        sec_info[field] = None
                                else:
                                    sec_info[field] = None
                                    
                        data.append(sec_info)
            
            if ev.eventType() == blpapi.Event.RESPONSE:
                break
                
        # Convert to DataFrame with nested data
        df = pd.DataFrame(data)
        
        # Save to CSV (note: nested data will be stored as strings)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"bulk_data_{timestamp}.csv"
        df.to_csv(filename, index=False)
        logger.info(f"Bulk data saved to {filename}")
        
        return df
        
    except Exception as e:
        logger.error(f"Error getting bulk field info: {str(e)}")
        return pd.DataFrame()

In [18]:
# Example usage of both regular and bulk data
analyzer = BloombergAnalyzer()

try:
    if analyzer.connect():
        securities = ["CSPX LN Equity"]
        
        # Get regular fields
        regular_data = analyzer.get_bulk_fields_info_v2(securities)
        print("\nRegular Field Data:")
        print(regular_data.head())
        
        # Get bulk fields
        bulk_data = analyzer.get_bulk_fields_info_v2(securities)
        
        # Display nested data
        print("\nBulk Field Data Example:")
        if not bulk_data.empty:
            for field in bulk_data.columns:
                if isinstance(bulk_data[field].iloc[0], list):
                    print(f"\n{field}:")
                    print(pd.DataFrame(bulk_data[field].iloc[0]))
        
finally:
    analyzer.disconnect()

INFO:__main__:Successfully connected to Bloomberg


AttributeError: 'BloombergAnalyzer' object has no attribute 'get_bulk_fields_info_v2'

In [21]:
# Example usage
analyzer = BloombergAnalyzer()

try:
    if analyzer.connect():
        securities = ["CSPX LN Equity"]
        
        # Get bulk data with proper structure
        data = analyzer.get_bulk_fields_info_v2(securities)
        
        # Display processed data
        print("\nProcessed Bulk Data:")
        
        # Show allocations
        for field, df in data['allocations'].items():
            print(f"\n{field}:")
            print(df.head())
            print("Total allocation:", df['Weight'].sum())
            
        # Show holdings
        for field, df in data['holdings'].items():
            print(f"\n{field}:")
            print(df.head())
            
        # Show fixed income data
        for field, df in data['fixed_income'].items():
            print(f"\n{field}:")
            print(df.head())
            
finally:
    analyzer.disconnect()

INFO:__main__:Successfully connected to Bloomberg
INFO:__main__:Sending bulk data request for 1 securities
INFO:__main__:Saved HB_INDUSTRY_SECTOR_ALLOCATION data to data/hb_industry_sector_allocation_20250121_185209.csv
INFO:__main__:Saved FUND_SECTOR_ALLOCATION data to data/fund_sector_allocation_20250121_185209.csv
INFO:__main__:Saved FUND_GEO_ALLOCATION data to data/fund_geo_allocation_20250121_185209.csv



Processed Bulk Data:

HB_INDUSTRY_SECTOR_ALLOCATION:
         Security
0  CSPX LN Equity
1  CSPX LN Equity
2  CSPX LN Equity
3  CSPX LN Equity
4  CSPX LN Equity


KeyError: 'Weight'